In [13]:
from typing import List
from concurrent.futures import ThreadPoolExecutor
import anthropic

from utils import ANTHROPIC_API_KEY

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

In [14]:
def complete(
    prompt: str, 
    sys_prompt: str = "",
    prefill: str = "",
    model_name: str = "claude-3-5-haiku-latest",
    temperature: float = 0.0,
    max_tokens: int = 4000,
    
):
    messages = [{"role": "user", "content": prompt}]
    if prefill != "":
        messages.append({"role": "assistant", "content": prefill})

    message = client.messages.create(
        model=model_name,
        max_tokens=max_tokens,
        temperature=temperature,
        system=sys_prompt,
        messages=messages
    )
    return message.content[0].text

### Example 1: Chain workflow

This workflow is ideal for situations where the task can be easily and cleanly decomposed into fixed subtasks. The main goal is to trade off latency for higher accuracy, by making each LLM call an easier task.

Each step progressively transforms raw text into a formatted table.

In [15]:
def chain(inp, prompts: List[str], llm_fn):
    result = inp
    for i, prompt in enumerate(prompts, 1):
        formatted_prompt = f"{prompt}\n\n Input: {result}"
        result = llm_fn(formatted_prompt)
    return result

In [16]:
data_processing_steps = [
    """Extract only the numerical values and their associated metrics from the text.
    Format each as 'value: metric' on a new line.
    Example format:
    92: customer satisfaction
    45%: revenue growth""",
    
    """Convert all numerical values to percentages where possible.
    If not a percentage or points, convert to decimal (e.g., 92 points -> 92%).
    Keep one number per line.
    Example format:
    92%: customer satisfaction
    45%: revenue growth""",
    
    """Sort all lines in descending order by numerical value.
    Keep the format 'value: metric' on each line.
    Example:
    92%: customer satisfaction
    87%: employee satisfaction""",
    
    """Format the sorted data as a markdown table with columns:
    | Metric | Value |
    |:--|--:|
    | Customer Satisfaction | 92% |"""
]

report = """
Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.
"""

print("\nInput text:")
print(report)
formatted_result = chain(report, data_processing_steps, complete)
print(formatted_result)


Input text:

Q3 Performance Summary:
Our customer satisfaction score rose to 92 points this quarter.
Revenue grew by 45% compared to last year.
Market share is now at 23% in our primary market.
Customer churn decreased to 5% from 8%.
New user acquisition cost is $43 per user.
Product adoption rate increased to 78%.
Employee satisfaction is at 87 points.
Operating margin improved to 34%.

Here's the data formatted as a markdown table:

| Metric | Value |
|:--|--:|
| Customer Satisfaction | 92% |
| Employee Satisfaction | 87% |
| Product Adoption Rate | 78% |
| Revenue Growth | 45% |
| User Acquisition Cost | 43.00 |
| Operating Margin | 34% |
| Market Share | 23% |
| Customer Churn | 5% |


### Example 2: Parallelization

Parallelization is effective when the divided subtasks can be parallelized for speed, or when multiple perspectives or attempts are needed for higher confidence results. For complex tasks with multiple considerations, LLMs generally perform better when each consideration is handled by a separate LLM call, allowing focused attention on each specific aspect.

In this example, try applying `func` per `something`.

Example:
- Create sports news letter article for all games played today.
- Create .... x per y

In [19]:
def mapped(prompt, inputs: List[str], llm_fn):
    raise NotImplementedError("fill me")

In [ ]:
def llm_call(inp):
    sys_prompt = "You are intelligent marketer!"
    return complete(inp, sys_prompt)

task_prompt = "" # fill me

results = mapped(
    task_prompt,
    [
        "" # fill me
        "", # fill me
    ],
    llm_call
)

for result in results:
    print(result)